#### Merging games dataframe and season stats dataframe

> Run all cells in sequence<br>
> Assumes season stats & game stats files are in '..\data' directory

In [10]:
import pandas as pd
import numpy as np
import os
from pathlib import Path
from tqdm import tqdm

In [2]:
full_path = os.getcwd()
base_path = str(Path(full_path).parents[0])  # one dir up
base_path = os.path.join(base_path, 'data')

team_stats_file = os.path.join(base_path, '1970-2020.csv')
team_schedules_file = os.path.join(base_path, '1970-2020_team_schedule.csv')

---

In [5]:
df_stats = pd.read_csv(team_stats_file, sep=';')
df_schedules = pd.read_csv(team_schedules_file, sep=';')

In [8]:
display(df_schedules.head())
display(df_stats.head())

,Unnamed: 0,Unnamed: 1,Unnamed: 2,game_day_of_week,game_date,game_time,boxscore_word,game_outcome,overtime,team_record,...,rush_yds_off,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,exp_pts_off,exp_pts_def,exp_pts_st
0,2019,New England Patriots,0,Sun,September 8,8:20PM ET,boxscore,W,NaN,1-0,...,99.0,NaN,15.0,308.0,276.0,32.0,1.0,18.24,9.50,-0.80
1,2019,New England Patriots,1,Sun,September 15,1:00PM ET,boxscore,W,NaN,2-0,...,124.0,1.0,11.0,184.0,142.0,42.0,4.0,14.12,35.44,-12.05
2,2019,New England Patriots,2,Sun,September 22,1:00PM ET,boxscore,W,NaN,3-0,...,68.0,2.0,6.0,105.0,69.0,36.0,1.0,1.65,26.43,-9.81
3,2019,New England Patriots,3,Sun,September 29,1:00PM ET,boxscore,W,NaN,4-0,...,74.0,1.0,23.0,375.0,240.0,135.0,4.0,-14.59,12.71,6.09
4,2019,New England Patriots,4,Sun,October 6,1:00PM ET,boxscore,W,NaN,5-0,...,130.0,1.0,11.0,223.0,78.0,145.0,2.0,9.23,20.78,-6.54


,Unnamed: 0,Unnamed: 1,team,g,drives,play_count_tip,score_pct,turnover_pct,plays_per_drive,yds_per_drive,...,two_pt_md,two_pt_att,def_two_pt,xpm,xpa,fgm,fga,safety_md,scoring,points_per_g
0,2019,New England Patriots,New England Patriots,16,190.0,1129.0,36.3,7.4,5.9,29.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019,Buffalo Bills,Buffalo Bills,16,186.0,1046.0,30.1,10.2,5.6,28.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019,New York Jets,New York Jets,16,189.0,984.0,23.3,11.6,5.2,22.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019,Miami Dolphins,Miami Dolphins,16,182.0,1052.0,30.2,13.7,5.8,27.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019,Baltimore Ravens,Baltimore Ravens,16,166.0,1093.0,51.8,8.4,6.6,39.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
unnamed_cols = {
    'Unnamed: 0': 'year',
    'Unnamed: 1': 'team',
    'Unnamed: 2': 'week_number'
}
df_stats.drop('Unnamed: 1', axis=1, inplace=True)
df_stats.rename(columns=unnamed_cols, inplace=True)
df_schedules.rename(columns=unnamed_cols, inplace=True)

#### add 'is_playoff' col

In [11]:
df_schedules['is_playoff'] = False

for season in tqdm(df_schedules['year'].unique()):
    df_season = df_schedules.query('year == @season')  # only current season
    # print(season, end=', ', flush=False)
    for team in df_season['team'].unique():
        df_season_team = df_season.query('team == @team')  # only current team
        try:  # try to find a row with 'playoff' value
            playoff_index = df_season_team[df_season_team['game_date'] == 'Playoffs'].index[0]
        except IndexError:
            playoff_index = False

        max_index = df_season_team.index[-1]
        if playoff_index:  # if there is a playoff row
            for row in range(playoff_index + 1, max_index + 1):  # for games after playoff row and before last row of current team and current season
                # playoff_game = df_season_team.loc[row].to_dict()
                
                df_schedules.loc[row, 'is_playoff'] = True

print('Done')

100%|███████████████████████████████████████████████████████████████████████████████████| 51/51 [00:02<00:00, 17.05it/s]

Done


In [12]:
df_schedules[df_schedules['is_playoff'] == True][['game_date', 'year', 'team', 'opp', 'game_date', 'game_outcome']]

,game_date,year,team,opp,game_date,game_outcome
18,January 4,2019,New England Patriots,Tennessee Titans,January 4,L
88,January 11,2019,Baltimore Ravens,Tennessee Titans,January 11,L
107,January 4,2019,Tennessee Titans,New England Patriots,January 4,W
108,January 11,2019,Tennessee Titans,Baltimore Ravens,January 11,W
109,January 19,2019,Tennessee Titans,Kansas City Chiefs,January 19,L
...,...,...,...,...,...,...
25872,January 4,1975,Oakland Raiders,Pittsburgh Steelers,January 4,L
25916,December 28,1975,Minnesota Vikings,Dallas Cowboys,December 28,L
25988,December 27,1975,St. Louis Cardinals,Los Angeles Rams,December 27,L
26060,December 27,1975,Los Angeles Rams,St. Louis Cardinals,December 27,W


#### remove 'bye week' & 'playoff' rows

In [13]:
df_bye_week = df_schedules[(df_schedules['opp'] == 'Bye Week') | (df_schedules['game_date'] == 'Playoffs')]
df_schedules = df_schedules.drop(df_bye_week.index, axis=0)

---

#### column cleaning

##### home & away games

In [14]:
home_games = df_schedules[df_schedules['game_location'] == '@'].index
away_games = df_schedules[pd.isna(df_schedules['game_location'])].index

df_schedules.loc[home_games, 'game_location'] = 'home'
df_schedules.loc[away_games, 'game_location'] = 'away'

df_schedules['game_location'].head()

0    away
1    home
2    away
3    home
4    home
Name: game_location, dtype: object

##### games with overtime

In [15]:
ot_games = df_schedules[df_schedules['overtime'] == 'OT'].index
non_ot_games = df_schedules[pd.isna(df_schedules['overtime'])].index

df_schedules.loc[ot_games, 'overtime'] = True
df_schedules.loc[non_ot_games, 'overtime'] = False

df_schedules['overtime'].head()

0    False
1    False
2    False
3    False
4    False
Name: overtime, dtype: object

#### further column cleaning

In [16]:
columns = df_schedules.columns
df_schedules = df_schedules.drop('boxscore_word', axis=1)

In [17]:
df_schedules.head()

,year,team,week_number,game_day_of_week,game_date,game_time,game_outcome,overtime,team_record,game_location,...,to_off,first_down_def,yards_def,pass_yds_def,rush_yds_def,to_def,exp_pts_off,exp_pts_def,exp_pts_st,is_playoff
0,2019,New England Patriots,0,Sun,September 8,8:20PM ET,W,False,1-0,away,...,NaN,15.0,308.0,276.0,32.0,1.0,18.24,9.50,-0.80,False
1,2019,New England Patriots,1,Sun,September 15,1:00PM ET,W,False,2-0,home,...,1.0,11.0,184.0,142.0,42.0,4.0,14.12,35.44,-12.05,False
2,2019,New England Patriots,2,Sun,September 22,1:00PM ET,W,False,3-0,away,...,2.0,6.0,105.0,69.0,36.0,1.0,1.65,26.43,-9.81,False
3,2019,New England Patriots,3,Sun,September 29,1:00PM ET,W,False,4-0,home,...,1.0,23.0,375.0,240.0,135.0,4.0,-14.59,12.71,6.09,False
4,2019,New England Patriots,4,Sun,October 6,1:00PM ET,W,False,5-0,home,...,1.0,11.0,223.0,78.0,145.0,2.0,9.23,20.78,-6.54,False


#### the actual merge

##### Column prefixes
    - gs_  : game_stat_
    - hts_ : home_team_stat_
    - ats_ : away_team_stat_

In [18]:
df_schedules.columns = 'gs_' + df_schedules.columns

df_home_team_stats = df_stats.copy()
df_home_team_stats.columns =  'hts_' + df_home_team_stats.columns

df_away_team_stats = df_stats.copy()
df_away_team_stats.columns =  'ats_' + df_away_team_stats.columns

In [20]:
df_games = pd.merge(
    df_schedules, 
    df_home_team_stats, 
    how='inner', 
    left_on=['gs_team', 'gs_year'], 
    right_on=['hts_team', 'hts_year'],
)
print('With home team stats:', df_games.shape)

df_games = pd.merge(
    df_games, 
    df_away_team_stats, 
    how='inner', 
    left_on=['gs_opp', 'gs_year'], 
    right_on=['ats_team', 'ats_year'],
)
print('With home and away team stats:', df_games.shape)

With home team stats: (24522, 66)
With home and away team stats: (24522, 105)


In [21]:
df_games[['gs_team', 'gs_opp', 'gs_year', 'gs_week_number', 'hts_wins', 'ats_wins']]

KeyError: "['hts_wins', 'ats_wins'] not in index"

In [28]:
import pickle

p = pickle.load(open(os.path.join('..', 'data', '1970-2020.pickle'), 'rb'))

In [35]:
p[0][2019]['New England Patriots'][16].keys()

dict_keys(['game_day_of_week', 'game_date', 'game_time', 'boxscore_word', 'game_outcome', 'overtime', 'team_record', 'game_location', 'opp', 'pts_off', 'pts_def', 'first_down_off', 'yards_off', 'pass_yds_off', 'rush_yds_off', 'to_off', 'first_down_def', 'yards_def', 'pass_yds_def', 'rush_yds_def', 'to_def', 'exp_pts_off', 'exp_pts_def', 'exp_pts_st'])

---

#### playoffs and regular season

In [ ]:
df_regular_season = df_games.query('gs_is_playoff == False')
df_playoffs = df_games.query('gs_is_playoff == True')

In [ ]:
df_regular_season.head()

In [ ]:
df_playoffs.head()

In [ ]:
game_stats_cols = df_playoffs.columns[pd.Series(df_playoffs.columns.str.startswith('gs_'))]
home_team_stats_cols = df_playoffs.columns[pd.Series(df_playoffs.columns.str.startswith('hts_'))]
away_team_stats_cols = df_playoffs.columns[pd.Series(df_playoffs.columns.str.startswith('ats_'))]

col_groups = [('game_stats', game_stats_cols), ('home_team_stats', home_team_stats_cols), ('away_team_stats', away_team_stats_cols)]

In [ ]:
for name, group in col_groups:
    c = 0
    print(f'{name}:', end=' ')
    for col in group:
        print(col, end=', ')
        c += 1
        if c == 5: break
    print('')

#### export to csv

In [ ]:
df_games.to_csv('all_games.csv', sep=';', encoding='utf-8', index=False)
df_playoffs.to_csv('playoffs.csv', sep=';', encoding='utf-8', index=False)
df_regular_season.to_csv('regular_season.csv', sep=';', encoding='utf-8', index=False)

---

#### exploration

##### 2020 & Kansas City Chiefs

In [ ]:
df_stats[(df_stats['year'] == 2020) & (df_stats['team'] == 'Kansas City Chiefs')]

In [ ]:
df_playoffs[(df_playoffs['gs_year'] == 2020) & (df_playoffs['gs_team'] == 'Kansas City Chiefs')][home_team_stats_cols]

##### 1970 & Dallas Cowboys

In [ ]:
df_stats[(df_stats['year'] == 1970) & (df_stats['team'] == 'Dallas Cowboys')]

In [ ]:
df_playoffs[(df_playoffs['gs_year'] == 1970) & (df_playoffs['gs_team'] == 'Dallas Cowboys')][home_team_stats_cols]